In [1]:
import numpy as np
import pandas as pd
from  scipy import sparse
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
 
train = pd.read_csv('linear_train.txt', names=['word', 'I'])
test = pd.read_csv('linear_test.txt', names=['word', 'I'])
sample_submission = pd.read_csv('linear_ans_example.txt')
y = train.as_matrix()[:, 1]
#X_words = train.as_matrix()[:, 0]
X_words = np.array(train['word'])
#print(X_words)

In [2]:
train.head(5)

,word,I
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


In [3]:
alph_size = 33
def word_vec_size(alph_size):
    return (alph_size**3 + alph_size**2 + alph_size)
size_f =  word_vec_size(alph_size)

In [12]:
alph_size = 33
def word_vec_size2(alph_size):
    return (alph_size**3 + alph_size**2 + alph_size + alph_size**2 + alph_size)
size_f2 =  word_vec_size(alph_size) + word_vec_size2(alph_size)

In [13]:
def lgram_encode(alph_size, word):
    word = word.lower()
    answ = (ord(word[0])-ord('а'))*alph_size**2 + (ord(word[1])-ord('а'))*alph_size + (ord(word[2])-ord('а'))
    return answ

In [14]:
def lgram_encode2(alph_size, word):
    word = word.lower()
    answ = (word_vec_size(alph_size) + (ord(word[0])-ord('а'))*alph_size + (ord(word[1])-ord('а')))
    return answ

In [15]:
def extract_2lgrams(alph_size, word, ind, data, ij):
    length = len(word)
    word_vec = np.zeros(word_vec_size2(alph_size))
    for i in range(length-2):
        lgram = word[i:i+2]
        lgram = lgram.lower()
        fail = 0
        for j in range(2):
            if(ord('а') > ord(lgram[j]) or ord(lgram[j]) > ord('я')):
                    fail = fail + 1
        if(fail == 0):            
            pos = lgram_encode2(alph_size, lgram)
            data.append(1)
            ij.append([ind, pos])

In [16]:
def extract_3lgrams(alph_size, word, ind, data, ij):
    length = len(word)
    word_vec = np.zeros(word_vec_size(alph_size))
    for i in range(length-2):
        lgram = word[i:i+3]
        lgram = lgram.lower()
        fail = 0
        for j in range(3):
            if(ord('а') > ord(lgram[j]) or ord(lgram[j]) > ord('я')):
                    fail = fail + 1
        if(fail == 0):            
            pos = lgram_encode(alph_size, lgram)
            data.append(1)
            ij.append([ind, pos])

In [17]:
print(size_f2)

75240


In [18]:
def make_features(train, alph_size, size):
    data = []
    ij = []
    for i in range(size):
        extract_3lgrams(alph_size, train[i], i, data, ij)
        extract_2lgrams(alph_size, train[i], i, data, ij)
    print(np.shape(data), np.shape(np.transpose(ij)))
    return(sparse.csr_matrix((data, np.transpose(ij)), shape=(size, size_f2)))

In [19]:
size = np.shape(X_words)[0]
X = make_features(X_words, 33, size)

(1299886,) (2, 1299886)


In [52]:
LG = linear_model.LogisticRegression(C = 0.4)

In [53]:
print(np.shape(X), np.shape(y))
print(y[:10])

(101408, 75240) (101408,)
[1 0 0 0 0 1 0 0 0 1]


In [54]:
LG.fit(X, np.array(train['I']))

LogisticRegression(C=0.4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
print(np.shape(X))

(101408, 75240)


In [56]:
roc_auc_score(np.array(train['I']), LG.predict_proba(X)[:, 1])

0.92062054868747079

In [57]:
modelcr=linear_model.LogisticRegression(C=0.4)

In [58]:
print(np.mean(model_selection.cross_val_score(modelcr, X, np.array(train['I']), scoring='roc_auc', cv=5)))

0.717968757072


In [59]:
X_test = np.array(test['word'])

In [60]:
size = np.shape(X_test)[0]
X_t = make_features(X_test, 33, size)

(2417614,) (2, 2417614)


In [61]:
print(np.shape(X_t))

(188920, 75240)


In [62]:
with open('linear_second.txt', 'w') as f:
    f.write('Id,Answer\n')
    iid = 0
    for iid, i in enumerate(LG.predict_proba(X_t)[:, 1]): # enumerate
        f.write(str(iid) + ',' + str(i) + '\n')
        iid += 1

In [63]:
s = []
with open('linear_second.txt', 'r') as f:
    for i in f:
        s.append(i)

In [64]:
s[1]

'0,0.368327043046\n'